In [1]:
import pandas as pd

In [2]:
CLICKS_FILE_PATH = '../data/raw/click_stream_new.csv'
CLICKS_OUTPUT_FILE_PATH = '../data/processed/one-day-clicks.csv'

TRANSACTIONS_FILE_PATH = '../data/raw/transaction_new.csv'
TRANSACTIONS_OUTPUT_FILE_PATH = '../data/processed/one-day-transactions.csv'

#### Find the most active day from the click_stream_new.csv file

In [3]:

def get_most_active_day(file_path):
    most_active_day = ''
    df_days_activities = pd.DataFrame()
    for df_chunck in pd.read_csv(file_path, chunksize=100000):
        df_chunck['event_date'] = [ d[:10] for d in df_chunck['event_time'] ]
        df_chunck['count'] = 0
        df_filtered = df_chunck[['event_date', 'count']]
        df_filtered_groupped = df_filtered.groupby(['event_date']).count()
        df_days_activities = pd.concat([df_days_activities, df_filtered_groupped], axis=0)
        
    df_days_activities.reset_index(inplace=True)
    
    df_days_groupped = df_days_activities.groupby(['event_date']).sum()
    df_days_groupped.reset_index(inplace=True)
    df_most_active_days = df_days_groupped[df_days_groupped['count'] == df_days_groupped['count'].max()]
    
    if len(df_most_active_days):
        most_active_day = list(df_most_active_days['event_date'])[0]
    return most_active_day

In [4]:
active_day = get_most_active_day(CLICKS_FILE_PATH)
print('The most active day was on: ', active_day)

The most active day was on:  2022-07-31


In [5]:
active_day

'2022-07-31'

#### Filter the click_stream_new.csv file to only keep records for that active day

In [6]:
df_clicks = pd.DataFrame()
for df_chunck in pd.read_csv(CLICKS_FILE_PATH, chunksize=100000):
    df_chunck['event_date'] = [ d[:10] for d in df_chunck['event_time'] ]
    df_filtered = df_chunck[df_chunck['event_date'].eq(active_day)]
    df_clicks = pd.concat([df_clicks, df_filtered], axis=0)


#### Generate ingestion time offsets

In [7]:
df_clicks['event_time'] = pd.to_datetime(df_clicks['event_time'])
df_clicks.sort_values(by='event_time', inplace=True)
df_clicks.reset_index(inplace=True)
df_clicks.head()

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_date
0,10164772,096fec88-b44c-4a27-ba14-48bcc1e670db,HOMEPAGE,2022-07-31 00:00:08.361823+00:00,881ee7c6-6d21-49c8-9e2f-e5cae9ea68e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31
1,10149939,12d8e1ff-a21b-4a6f-86f7-f5f31557986e,SCROLL,2022-07-31 00:00:19.191828+00:00,3057a1c1-828e-4d17-bf2c-a21292450613,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31
2,2321955,d7b4d7c4-17b3-48b7-92c3-73b484fd9f2f,SEARCH,2022-07-31 00:00:20.822075+00:00,649c483b-057b-4f5b-ac12-ac9e2ac216ff,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31
3,11963211,d5e152a1-e8d9-4d1a-a646-9ba7b31c5b96,SEARCH,2022-07-31 00:00:22.101671+00:00,9a060895-60e3-4ab3-8af0-566fb4f1dc9b,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31
4,10878721,04305057-f8b2-4265-bec1-8db695929418,SCROLL,2022-07-31 00:00:22.542968+00:00,7adf17f3-dece-4000-ae45-60c456504256,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31


In [8]:
df_clicks['event_offset'] = 0

for i, row in df_clicks.iterrows():
    if i > 0:
        df_clicks.at[i, 'event_offset'] = int((df_clicks.at[i, 'event_time'] - df_clicks.at[i-1, 'event_time']).total_seconds())

df_clicks.head()

,index,session_id,event_name,event_time,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_date,event_offset
0,10164772,096fec88-b44c-4a27-ba14-48bcc1e670db,HOMEPAGE,2022-07-31 00:00:08.361823+00:00,881ee7c6-6d21-49c8-9e2f-e5cae9ea68e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0
1,10149939,12d8e1ff-a21b-4a6f-86f7-f5f31557986e,SCROLL,2022-07-31 00:00:19.191828+00:00,3057a1c1-828e-4d17-bf2c-a21292450613,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,10
2,2321955,d7b4d7c4-17b3-48b7-92c3-73b484fd9f2f,SEARCH,2022-07-31 00:00:20.822075+00:00,649c483b-057b-4f5b-ac12-ac9e2ac216ff,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31,1
3,11963211,d5e152a1-e8d9-4d1a-a646-9ba7b31c5b96,SEARCH,2022-07-31 00:00:22.101671+00:00,9a060895-60e3-4ab3-8af0-566fb4f1dc9b,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,2022-07-31,1
4,10878721,04305057-f8b2-4265-bec1-8db695929418,SCROLL,2022-07-31 00:00:22.542968+00:00,7adf17f3-dece-4000-ae45-60c456504256,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-31,0


In [9]:
# the minimum click event time 
min_click_event_time = df_clicks['event_time'].min()

In [10]:
df_clicks = df_clicks[['session_id', 'event_name', 'event_id', 'traffic_source', 'product_id', 'quantity', 'item_price',
       'payment_status', 'search_keywords', 'promo_code', 'promo_amount', 'event_offset']]
df_clicks.head()

,session_id,event_name,event_id,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,event_offset
0,096fec88-b44c-4a27-ba14-48bcc1e670db,HOMEPAGE,881ee7c6-6d21-49c8-9e2f-e5cae9ea68e7,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,12d8e1ff-a21b-4a6f-86f7-f5f31557986e,SCROLL,3057a1c1-828e-4d17-bf2c-a21292450613,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
2,d7b4d7c4-17b3-48b7-92c3-73b484fd9f2f,SEARCH,649c483b-057b-4f5b-ac12-ac9e2ac216ff,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,1
3,d5e152a1-e8d9-4d1a-a646-9ba7b31c5b96,SEARCH,9a060895-60e3-4ab3-8af0-566fb4f1dc9b,MOBILE,NaN,NaN,NaN,NaN,T-Shirt,NaN,NaN,1
4,04305057-f8b2-4265-bec1-8db695929418,SCROLL,7adf17f3-dece-4000-ae45-60c456504256,MOBILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### Save prepared clicks data to a file 

In [11]:
df_clicks.to_csv(CLICKS_OUTPUT_FILE_PATH, index=False)

#### Filter the transaction_new.csv file to only keep records for the active day

In [12]:
df_transac = pd.read_csv(TRANSACTIONS_FILE_PATH)
df_transac.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753


In [13]:
df_transac['event_date'] = [ d[:10] for d in df_transac['created_at'] ]
df_transactions = df_transac[df_transac['event_date'].eq(active_day)]

In [14]:
df_transactions.sort_values(by='created_at', inplace=True) 
df_transactions.reset_index(inplace=True)

/tmp/ipykernel_36279/3260585774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions.sort_values(by='created_at', inplace=True)


#### Generate ingestion time offsets for transactions data

In [15]:
df_transactions['created_at'] = pd.to_datetime(df_transactions['created_at'])
df_transactions['event_offset'] = 0

for index, row in df_transactions.iterrows():
    if index == 0:
        df_transactions.at[index, 'event_offset'] = int((df_transactions.at[index, 'created_at'] - min_click_event_time).total_seconds()) 
    else:
        df_transactions.at[index, 'event_offset'] = int((df_transactions.at[index, 'created_at'] - df_transactions.at[index-1, 'created_at']).total_seconds())

df_transactions.head(5)

/tmp/ipykernel_36279/518682148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions['created_at'] = pd.to_datetime(df_transactions['created_at'])
/tmp/ipykernel_36279/518682148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions['event_offset'] = 0


,index,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,event_date,event_offset
0,271002,2022-07-31 00:01:57.082014+00:00,56249,8b814fb1-866f-490e-8a86-336a0ec07ce6,764e800f-0e4d-41f4-86b7-ac125e09f8f7,OVO,Success,0,NaN,10000,2022-08-02T01:02:41.871378Z,-6.760483,107.145038,234659,59595,1,224659,2022-07-31,108
1,943970,2022-07-31 00:02:30.514880+00:00,88526,afd64790-3605-4d36-b5b0-abd2e3a5823d,8bdc6d05-b7c2-422c-a74c-b636ae0bc660,Credit Card,Success,3881,WEEKENDSERU,10000,2022-08-02T20:33:03.125175Z,0.458978,117.285013,283036,17183,1,276917,2022-07-31,33
2,412052,2022-07-31 00:04:20.909479+00:00,41121,798a298b-2966-4d69-8b9a-fadc4f87b857,eacfe57e-6c19-413a-a3eb-4c79f94a1f33,Credit Card,Success,0,NaN,0,2022-08-04T07:20:51.990247Z,-6.186397,106.872607,92318,45127,1,92318,2022-07-31,110
3,865745,2022-07-31 00:06:17.377315+00:00,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,2022-08-02T17:44:45.837917Z,-0.275616,110.529582,527040,44184,1,247548,2022-07-31,116
4,865744,2022-07-31 00:06:17.377315+00:00,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,2022-08-02T17:44:45.837917Z,-0.275616,110.529582,527040,12913,1,279492,2022-07-31,0


In [16]:
df_transactions.columns

Index(['index', 'created_at', 'customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'shipment_date_limit', 'shipment_location_lat',
       'shipment_location_long', 'total_amount', 'product_id', 'quantity',
       'item_price', 'event_date', 'event_offset'],
      dtype='object')

In [17]:
df_transactions = df_transactions[['customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'shipment_location_lat', 'shipment_location_long', 'total_amount', 'product_id', 'quantity',
       'item_price', 'event_offset']]
df_transactions.head()

,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price,event_offset
0,56249,8b814fb1-866f-490e-8a86-336a0ec07ce6,764e800f-0e4d-41f4-86b7-ac125e09f8f7,OVO,Success,0,NaN,10000,-6.760483,107.145038,234659,59595,1,224659,108
1,88526,afd64790-3605-4d36-b5b0-abd2e3a5823d,8bdc6d05-b7c2-422c-a74c-b636ae0bc660,Credit Card,Success,3881,WEEKENDSERU,10000,0.458978,117.285013,283036,17183,1,276917,33
2,41121,798a298b-2966-4d69-8b9a-fadc4f87b857,eacfe57e-6c19-413a-a3eb-4c79f94a1f33,Credit Card,Success,0,NaN,0,-6.186397,106.872607,92318,45127,1,92318,110
3,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,-0.275616,110.529582,527040,44184,1,247548,116
4,14151,21656164-fa7c-43c7-a369-0fb05a980299,83a01b81-fe42-43bb-97d1-93244c847594,Gopay,Success,0,NaN,0,-0.275616,110.529582,527040,12913,1,279492,0


#### Save prepared transactions data to a file 

In [18]:
df_transactions.to_csv(TRANSACTIONS_OUTPUT_FILE_PATH, index=False)